# Summary of the results for Ge and Xe detectors om the migdal effect.

Below the following three tests are highlighted:
  - Earth shielding versus normal SHM
  - Profiling out astrophysical uncertainties as nuisance parameters
  - Comparing Ge vs. Xe. detectors.
  - Complementarity of Ge and Xe detecotors.

In [ ]:
# was 176

### TODO:
 - overlay with alpha <1
 - remove ommit_check = 'log' as it is bad
 - only use updated prior-ranges
 - plot complementarity of Ge/Xe
 - add method to base xlim, ylim on mean and std returning from _confidence_plot

In [ ]:
# Setup packages
%run load_results_multinest.py

In [ ]:
# Acces data by using dataframe called 'df'
df.head(2)

In [ ]:
# Let's write out a summary for the purpose of filling out missing gaps for sets of simulations that are missing
finished = df[['item','mw', 'config_sigma', 'config_detector', 'config_notes', 'n_fit_parameters', 'config_nlive', 'config_save_intermediate', 'config_earth_shielding']].copy()
def strip_n(n):
    if n.endswith('038'):  n = n[:-1]
    elif n.endswith('_n12') or n.endswith('_n16'): n = n[:-4]
    return n
notes = [strip_n(n) for n in finished.config_notes.values]
finished['config_notes'] = notes
finished.to_csv('../finished_sims.csv', index=False)
finished.head(3)

In [ ]:
# For this moment let's focus on the data from a specific set of simulations
# these all have the following note in their configuration file.
# why does this syntax not work?
# mask = (df['config_notes'] == 'migdal_04_038')  | (df['config_notes'] == 'migdal_04_03') 
mask = np.array(['migdal_04_03' in _n for _n in df['config_notes']])
df_copy = df[~mask].copy()
df_copy.head(2)

In [ ]:
np.sum(df['config_notes'] == 'migdal_04_038'), np.sum(df['config_notes'] == 'migdal_04_03')

In [ ]:
len(df_copy)

In [ ]:
df_copy = df.copy()
len(df_copy)

# Helper functions
Temporary to but will be added to py-file

#TODO

# Earth shielding
Below we compare the earth shielding versus non-earth shielding. To this end we first select a 'normal' dataset with a high cross-section for the given mass. We also list the other opportunities, one could make another selection. For presentation purposes not all scenarios are presented below.

Step by step:
  - Check for the mass that the SHM and verne results are available
  - List all masses -> get for each mass the largest cross-section
  - For that set of parameters plot the difference between verne and SHM 
  - Do the same for a superCDMS-type detector.
  

## Finding the data

In [ ]:
print('We want to comment out "tol" below')

In [ ]:
mask = (df_copy['config_detector'] == 'Xe_migd_bg') & (
    df_copy['config_earth_shielding'] == False)    & (
    df_copy['config_save_intermediate'] == False)
sel_df = df_copy[mask]
to_check, to_check_biggest = [], []
for j, m in enumerate(np.unique(sel_df.mw.values)):
    for k, s in enumerate(np.flip(np.sort(np.unique(sel_df[sel_df['mw']==m].config_sigma.values)))):
        print('Available for m = %.1f GeV/c2, s = 10^%i cm^2:'%(m,s))
        mask = (sel_df.mw == m) & (sel_df.config_sigma == s)
        items = sel_df[mask].item.values
        print(f'\t{len(sel_df[mask])} items, see i in {items}')

        for it in items:
            matched_to = match_other_item(it, verbose = True,
                                          ommit_matching=['config_save_intermediate',
                                                          'config_halo_model', 
                                                          'tol'],
                                          anti_match_for='config_earth_shielding')
            if type(matched_to) == np.int64:
                to_check.append([it, matched_to])
                if j == len(to_check_biggest): to_check_biggest.append([it, matched_to])
            elif np.iterable(matched_to) and np.iterable((matched_to[0])):
                print(f'multiple matches for {it} ({matched_to[0]}) taking {matched_to[0][0]}')
                to_check.append([it, matched_to[0][0]])
                if j == len(to_check_biggest): to_check_biggest.append([it, matched_to])
            else:
                print(f'\t{it} is matched to {matched_to} and/but omitted')
        if j == len(to_check_biggest):
            print('No mathes for m = %.1f GeV/c2, s = 10^%i cm^2\n'%(m,s))
            to_check_biggest.append([])
to_check, to_check_biggest

In [ ]:
to_check, to_check_biggest

## Plot the matches

In [ ]:
# Look at the matches where the cross-section is the biggeest
for its in to_check_biggest:
    for it in its:
        one_confidence_plot(it, corner=False)
        plt.grid(axis='y')
        plt.show()

In [ ]:
# Or have a look at a random selection of arbitrary cross-sections
random_set = np.random.randint(0,len(to_check))
# for its in to_check:
for its in to_check[random_set:random_set+1]:
    for it in its:
        one_confidence_plot(it, corner=False)
        plt.grid(axis='y')
        plt.show()

# Astrophysical uncertainties

Below we load a set of data where we compare fits with the same fits where the astrophysical uncertainties are treated as astrophysical parameters

In [ ]:
mask = (df_copy['config_detector'] == 'Xe_migd_bg') & (
    df_copy['config_earth_shielding'] == False) & (
    df_copy['n_fit_parameters'] == 2)

sel_df = df_copy[mask]
to_check, to_check_lowest = [], []
for j, m in enumerate(np.unique(sel_df.mw.values)):
    for k, s in enumerate(np.sort(np.unique(sel_df[sel_df['mw']==m].config_sigma.values))):
        print('Available for m = %.1f GeV/c2, s = 10^%i cm^2:'%(m,s))
        mask = (sel_df.mw == m) & (sel_df.config_sigma == s)
        items = sel_df[mask].item.values
        print(f'\t{len(sel_df[mask])} items, see i in {items}')

        for it in items:
            matched_to = match_other_item(it, verbose = False,
                                          ommit_matching=['config_save_intermediate',
                                                         'tol'],
                                          anti_match_for='n_fit_parameters')
            if type(matched_to) == np.int64:
                to_check.append([it, matched_to])
                if j == len(to_check_lowest): to_check_lowest.append([it, matched_to])
            elif np.iterable(matched_to) and np.iterable((matched_to[0])):
                print(f'multiple matches for {it} ({matched_to[0]}) taking {matched_to[0][0]}')
                to_check.append([it, matched_to[0][0]])
                if j == len(to_check_lowest): to_check_lowest.append([it, matched_to[0][0]])
            else:
                print(f'\t{it} is matched to {matched_to} and/but omitted')
        if j == len(to_check_lowest):
            print('No mathes for m = %.1f GeV/c2, s = 10^%i cm^2\n'%(m,s))
            to_check_lowest.append([])
to_check, to_check_lowest

In [ ]:
to_check, to_check_lowest

In [ ]:
for its in to_check_lowest:
    print(its)
    for it in its:
        one_confidence_plot(it, corner=False)
        plt.grid(axis='y')
        plt.show()
    if len(its):
        overlay_confidence_plots(its)
        plt.show()

In [ ]:
# Or have a look at a random selection of arbitrary cross-sections
random_set = np.random.randint(0,len(to_check))
# for its in to_check:
for its in to_check[random_set:random_set+1]:
    print(its)
    for it in its:
        one_confidence_plot(it, corner=False)
        plt.grid(axis='y')
#         plt.xlim(-.2, 0.2)
#         plt.ylim(-36.5, -35.5)

        plt.show()
    if len(its):
        plt.figure(figsize=(10,6))
        for k, i in enumerate(its):
            _one_confidence_plot(i, smoothing=True, nsigma=3, nbins = 200, cmap = [cm.viridis, cm.gnuplot2_r][k])
#         plt.xlim(-.2, 0.2)
#         plt.ylim(-36.5, -35.5)
        plt.grid(axis='y')

        plt.show()

# Comparing Ge vs. Xe. detectors.

In [ ]:
mask = (df_copy['config_detector'] == 'Ge_migd_HV_bg')

sel_df = df_copy[mask]
to_check = []
for j, m in enumerate(np.unique(sel_df.mw.values)):
    for k, s in enumerate(np.sort(np.unique(sel_df[sel_df['mw']==m].config_sigma.values))):
        print('Available for m = %.1f GeV/c2, s = 10^%i cm^2:'%(m,s))
        mask = (sel_df.mw == m) & (sel_df.config_sigma == s)
        items = sel_df[mask].item.values
        print(f'\t{len(sel_df[mask])} items, see i in {items}')

        for it in items:
            print('WARNING OMMITTING prior_log* in compare!!!')
            matched_to_Xe = match_other_item(it, verbose = False,
                                          diff_det_type=True,
                                          ommit_matching=['config_save_intermediate',
#                                                           'prior_log*',
                                                          'tol'
                                                         ],
                                          anti_match_for='config_detector')
            matched_to_Ge = match_other_item(it, verbose = False,
                                          diff_det_type=False,
                                          ommit_matching=['config_save_intermediate',
                                                          'tol'
                                                         ],
                                          anti_match_for='config_detector')
            if matched_to_Xe and matched_to_Ge:
                to_check.append([it])
                for _m in [matched_to_Ge, matched_to_Xe]:
                    if type(_m) == np.int64:
                        to_check[-1].append(_m)
                    else:
                        try:
                            _m2 = _m[0][0]
                            print(f'Trying to append 0,0 of m {_m2}')
                            if type(_m2) == np.int64:
                                to_check[-1].append(_m2)
                        except Exception as e:
                            raise e
                            print(f'{_m}, {e}')
                if len(to_check[-1]) != 3:
                    print(f'removing stuff for {it} as it does not have three succesfull matches')
            else:
                print(f'No match for both other detectortypes for {it}. Only found {[m[0] for m in [matched_to_Xe, matched_to_Ge] if np.iterable(m)]}')
to_check

In [ ]:
to_check

In [ ]:
# Or have a look at a random selection of arbitrary cross-sections
random_set = np.random.randint(0,len(to_check))
for its in to_check:
# for its in [[167, 37, 113]]:
# for its in to_check[random_set:random_set+1]:
    print('-----------------------------------------------------------------------------')
    print(its)
    for it in its:
        one_confidence_plot(it, corner=False)
        plt.grid(axis='y')
        plt.show()
    plt.figure(figsize=(12,6))
    three_confidence_plot(its[:2])
    
    if len(its) == 3:
        plt.figure(figsize=(10,6))
        two_confidence_plot(its[:2],nsigma =2)

        for k, i in enumerate(its):
            if k<2: continue
            _one_confidence_plot(i, smoothing=True, nsigma=3, nbins = 200, cmap = [cm.viridis, cm.gnuplot2_r,
                                                                                  cm.gist_heat][k])
        plt.grid(axis='y')

        plt.show()